In [29]:
import pandas as pd
import openai
import time
import re
from scipy import stats
from sklearn.metrics import mean_absolute_error
import pickle
import numpy as np


api_key = open('openai.key', 'r').read()
openai.api_key = api_key

# Prompt Preperation and Helper Methods

Description from paper: 
- The participants in this study were first asked to stand up and focus on different aspects of their bodily experience of standing in the same way as did participants in Experiment 1.
- Following this, we introduced the five image schemas of BALANCE, VERTICALITY, CENTER-PERIPHERY, RESISTANCE, and LINKAGE as was done in Experiment 1

In [3]:
definitions = {
    "VERTICALITY": "Consider the notion of VERTICALITY. Verticality refers to the sense of an extension along an up—down orientation.",
    "BALANCE": "Consider the notion of BALANCE. Balance refers to your sense of symmetry or stability relative to some point within your body.",
    "CENTER-PERIPHERY": "Consider the notion of CENTER-PERIPHERY. Center-periphery refers to the experience of some objects or events as central while surrounding objects and events are peripheral or to the outside.",
    "LINKAGE": "Consider the notion of LINKAGE. Linkage refers to the perception of a connection between objects or events.",
    "RESISTANCE": "Consider the notion of RESISTANCE. Resistance refers to the experience of your body opposing some external force."
}

In [4]:
exp_3_df = pd.read_csv('../../data/Gibbs_Exp3.csv', sep=';')
# drop nan rows
exp_3_df = exp_3_df.dropna()
exp_4_df = pd.read_csv('../../data/Gibbs_Exp4.csv', sep=';')
stimuli_exp3 = exp_3_df['Stimulus'].values
stimuli_exp4 = exp_4_df['Stimulus'].values
print(len(stimuli_exp3))
print(len(stimuli_exp4))

32
32


In [50]:
# create prompts
prompts_exp3 = {"VERTICALITY":[], "BALANCE":[], "CENTER-PERIPHERY":[], "LINKAGE":[], "RESISTANCE":[]}
prompts_exp4 = {"VERTICALITY":[], "BALANCE":[], "CENTER-PERIPHERY":[], "LINKAGE":[], "RESISTANCE":[]}


for i in range(len(stimuli_exp3)):
    prompt_ending =  "How strongly is the phrase ''" + stimuli_exp3[i] + "'' related to this notion on a scale from 1 (not at all related) to 7 (very strongly related)?"
    prompts_exp3["VERTICALITY"].append(definitions["VERTICALITY"] + "\n" + prompt_ending)
    prompts_exp3["BALANCE"].append(definitions["BALANCE"] + "\n" + prompt_ending)
    prompts_exp3["CENTER-PERIPHERY"].append(definitions["CENTER-PERIPHERY"] + "\n" + prompt_ending)
    prompts_exp3["LINKAGE"].append(definitions["LINKAGE"] + "\n" + prompt_ending)
    prompts_exp3["RESISTANCE"].append(definitions["RESISTANCE"] + "\n" + prompt_ending)

for i in range(len(stimuli_exp4)):
    prompt_ending =  "How strongly is the phrase ''" + stimuli_exp4[i] + "'' related to this notion on a scale from 1 (not at all related) to 7 (very strongly related)?"
    prompts_exp4["VERTICALITY"].append(definitions["VERTICALITY"] + "\n" + prompt_ending)
    prompts_exp4["BALANCE"].append(definitions["BALANCE"] + "\n" + prompt_ending)
    prompts_exp4["CENTER-PERIPHERY"].append(definitions["CENTER-PERIPHERY"] + "\n" + prompt_ending)
    prompts_exp4["LINKAGE"].append(definitions["LINKAGE"] + "\n" + prompt_ending)
    prompts_exp4["RESISTANCE"].append(definitions["RESISTANCE"] + "\n" + prompt_ending)

In [95]:
# variants with which GPT can be prompted to generate the questionnaire answers
def gpt_complete(prompt, model):
    temperature = 0
    return openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=100,
        temperature=temperature
    ) 

def gpt_complete_proba(prompt, model):
    # get the probability distribution over the next token for the tokens 1, 2, 3, 4, 5, 6, 7
    temperature = 0
    results = []
    debug = []
    for i in range(1, 8):
        next_prompt = prompt + "\n\n"+ str(i)
        print(next_prompt)
        result = openai.Completion.create(
            model=model,
            prompt=next_prompt,
            max_tokens=1,
            temperature=temperature,
            logprobs=1,
            echo=True
        )
        debug.append(result)
        #print(result)
        # the index of i in the list of tokens
        index = result["choices"][0]["logprobs"]["tokens"].index(str(i))
        print(index)
        print(result["choices"][0]["logprobs"]["token_logprobs"][index])
        print(np.exp(result["choices"][0]["logprobs"]["token_logprobs"][index]))
        print(result["choices"][0]["logprobs"]["tokens"][index])
        print()
        log_prob_result=result["choices"][0]["logprobs"]["token_logprobs"][index]
        prob_result = np.exp(log_prob_result)
        results.append(prob_result)
        # wait 20 seconds to avoid exceeding the API rate limit
        time.sleep(21)
    average_completion = 0
    for i in range(1, 8):
        average_completion += i * results[i-1]
    average_completion = average_completion / sum(results)      # TODO: verify this is the right operation

    return results, average_completion, debug


# incomplet methods: 
def gpt_complete_reasoning(prompt, model):
    prompt += "Reason step by step: \n"
    pass

def gpt_with_history(prompt, model):
    pass

def gpt_simulate_human(prompt, model, name, gender):           # TE paradigm 
    prompt = name + " was asked to..." + name +" responded with the number: " + prompt
    pass

In [92]:
# example prompt
prompts_exp3["VERTICALITY"][0]

"Consider the notion of VERTICALITY. Verticality refers to the sense of an extension along an up—down orientation.\nHow strongly is the phrase ''stand at attention'' related to this notion on a scale from 1 (not at all related) to 7 (very strongly related)?"

In [97]:
# test
model = "text-davinci-003"

probas,r,d = gpt_complete_proba(prompt=prompts_exp3["VERTICALITY"][0], model=model)
print(probas)
print(r)


Consider the notion of VERTICALITY. Verticality refers to the sense of an extension along an up—down orientation.
How strongly is the phrase ''stand at attention'' related to this notion on a scale from 1 (not at all related) to 7 (very strongly related)?

1
60
-19.92949
2.2117318185277327e-09
1

Consider the notion of VERTICALITY. Verticality refers to the sense of an extension along an up—down orientation.
How strongly is the phrase ''stand at attention'' related to this notion on a scale from 1 (not at all related) to 7 (very strongly related)?

2
60
-20.815136
9.122235821808137e-10
2

Consider the notion of VERTICALITY. Verticality refers to the sense of an extension along an up—down orientation.
How strongly is the phrase ''stand at attention'' related to this notion on a scale from 1 (not at all related) to 7 (very strongly related)?

3
60
-19.669903
2.8672771090706086e-09
3

Consider the notion of VERTICALITY. Verticality refers to the sense of an extension along an up—down orie

In [58]:
# test
model = "text-davinci-003"

probas,r,d = gpt_complete_proba(prompt=prompts_exp3["VERTICALITY"][1], model=model)
print(probas)
print(r)


[0.4693367739968185, 0.6304976189698971, 0.0021300609172966016, 0.0004053786456902955, 0.00030428463356096594, 2.8846006426974795e-05, 4.566600513491415e-06]
1.5779979076082327


In [87]:
d[0]["choices"][0]["logprobs"]["tokens"].index("1")

61

In [34]:
log_prob=c[0]["choices"][0]["logprobs"]["token_logprobs"][-2]
# translate log_prob to prob
prob = np.exp(log_prob)
print(prob)

0.9953776823446714


# Generating the Data with LLMs

In [86]:
model = "text-davinci-003"
completions_exp3 = {"VERTICALITY": [], "BALANCE": [], "CENTER-PERIPHERY": [], "LINKAGE": [], "RESISTANCE": []}

for i in range(len(stimuli_exp3)):
    for k in completions_exp3:
        completions_exp3[k].append(gpt_complete(prompts_exp3[k][i], model))
        time.sleep(30)

with open('completions_exp3.pkl', 'wb') as f:
    pickle.dump(completions_exp3, f)

RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-3JxRZ6OWspj2q2bxuxmCUL9a on requests per day. Limit: 200 / day. Please try again in 7m12s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.

In [58]:
completions_exp4 = {"VERTICALITY": [], "BALANCE": [], "CENTER-PERIPHERY": [], "LINKAGE": [], "RESISTANCE": []} 
assert len(completions_exp3) == len(completions_exp4)
for i in range(len(stimuli_exp4)):
    for k in completions_exp4:
        completions_exp4[k].append(gpt_complete(prompts_exp4[k][i], model))
        time.sleep(30)

with open('completions_exp4.pkl', 'wb') as f:
    pickle.dump(completions_exp4, f)

In [60]:
# extract only the numbers from the completions

completions_exp3_clean = {"VERTICALITY": [], "BALANCE": [], "CENTER-PERIPHERY": [], "LINKAGE": [], "RESISTANCE": []}
completions_exp4_clean = {"VERTICALITY": [], "BALANCE": [], "CENTER-PERIPHERY": [], "LINKAGE": [], "RESISTANCE": []}

for k in completions_exp3:
    for i in range(len(completions_exp3[k])):
        clean_item = completions_exp3[k][i]["choices"][0]["text"]
        clean_item = int(re.findall(r'\d+', clean_item)[0])
        completions_exp3_clean[k].append(clean_item)

for k in completions_exp4:
    for i in range(len(completions_exp4[k])):
        clean_item = completions_exp4[k][i]["choices"][0]["text"]
        clean_item = int(re.findall(r'\d+', clean_item)[0])
        completions_exp4_clean[k].append(clean_item)

# Analysing the Data

## Correlation Analysis

In [85]:
print("Spearman correlations for experiment 3")
for k in completions_exp3_clean:
    print(k, stats.spearmanr(completions_exp3_clean[k], exp_3_df[k]))

print("Spearman correlations for experiment 4")
for k in completions_exp4_clean:
    print(k, stats.spearmanr(completions_exp4_clean[k], exp_4_df[k]))
    

Spearman correlations for experiment 3
Spearman correlations for experiment 4
VERTICALITY SpearmanrResult(correlation=0.6833876221034471, pvalue=1.627063547884175e-05)
BALANCE SpearmanrResult(correlation=0.6771840725179004, pvalue=2.0761491611577507e-05)
CENTER-PERIPHERY SpearmanrResult(correlation=0.6901380535614554, pvalue=1.23965800957632e-05)
LINKAGE SpearmanrResult(correlation=0.5197179828067287, pvalue=0.0022993844979337305)
RESISTANCE SpearmanrResult(correlation=0.7058717439215124, pvalue=6.389977181110315e-06)


In [21]:
r_V = stats.spearmanr(completion_numbers_V, exp_3_df["V"].values)
print("Verticality Spearman R: ", r_V)

r_B = stats.spearmanr(completion_numbers_B, exp_3_df["B"].values)
print("Balance Spearman R: ", r_B)

r_C = stats.spearmanr(completion_numbers_C, exp_3_df["CP"].values)
print("Center-Peripher Spearman R: ", r_C)

r_R = stats.spearmanr(completion_numbers_R, exp_3_df["R"].values)
print("Resistance Spearman R: ", r_R)

r_L = stats.spearmanr(completion_numbers_L, exp_3_df["L"].values)
print("Linkage Spearman R: ", r_L)

Verticality Spearman R:  SpearmanrResult(correlation=0.5319115370624121, pvalue=0.001728905163310417)
Blockage Spearman R:  SpearmanrResult(correlation=0.3889504346025759, pvalue=0.02779617841811564)
Center-Peripher Spearman R:  SpearmanrResult(correlation=0.7397528813626304, pvalue=1.3126737552856912e-06)
Resistance Spearman R:  SpearmanrResult(correlation=0.6820014752671762, pvalue=1.71900059033538e-05)
Linkage Spearman R:  SpearmanrResult(correlation=0.5431252313598959, pvalue=0.0013174767080267854)


In [28]:
# Mean Absolute Error
print("MAE Verticality:", mean_absolute_error(completion_numbers_V, exp_3_df['V']))
print("MAE Balance:", mean_absolute_error(completion_numbers_B, exp_3_df['B']))
print("MAE CP:", mean_absolute_error(completion_numbers_C, exp_3_df['CP']))
print("MAE Resistance:", mean_absolute_error(completion_numbers_R, exp_3_df['R']))
print("MAE Linkage:", mean_absolute_error(completion_numbers_L, exp_3_df['L']))






MSE Verticality: 1.8465624999999999
MSE Balance: 1.448125
MSE CP: 2.0453125
MSE Resistance: 1.2834375
MSE Linkage: 2.364375


In [ ]:
# make new csv 

## Linear Discriminant Analysis

We repeate the analysis from the paper but this time with the numbers generated from GPT instead of humans

Problem: The paper scores can not be reproduced!

In [146]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis 

from sklearn.metrics import accuracy_score

In [105]:
cluster_1=["one-night stand", "let the mixture stand", "a standing ovation", "on the witness stand", "the police told them to stand back", "stand at attention", "let the issue stand", "stand out in several sports", "the boss always stands over me", "to stand the test of time", "don't stand for such treatment"]
cluster_2=["to stand in someone else's shoes", "the clock stands on the mantle", "he stands committed", "we stand on 30 years of experience", "united we stand", "stand on shaky ground", "to stand firm", "stand in awe", "to stand to profit"]
cluster_3=["stand by your man", "to stand accused", "the house stands in the clearing", "he stands six-foot nine", "the barometer stands at 30 inches", "to stand against great odds", "they did nothing but stand around", "the engine can't stand the constant wear", "the part stands for the whole", "it stands to reason", "get stood up for a date", "as the matter now stands"]

In [149]:
# LDA with GPT data 
y = []
for s in stimuli_exp3:
    if s in cluster_1:
        y.append(0)
    elif s in cluster_2:
        y.append(1)
    elif s in cluster_3:
        y.append(2)
    else:
        print('error regarding:', s)

x_exp3 = []
x_exp4 = []
for i in range(len(stimuli_exp4)):
    schema_profile = []
    for k in completions_exp4_clean:
        schema_profile.append(completions_exp4_clean[k][i])
    x_exp4.append(schema_profile)


clf = QuadraticDiscriminantAnalysis()
clf.fit(x_exp4, y)
y_pred4 = clf.predict(x_exp4)
print('accuracy exp4:', accuracy_score(y_true=y, y_pred=y_pred4))





accuracy exp4: 0.84375


In [147]:
# LDA with original participant data
x_original3 = []
x_original4 = []
for s in stimuli_exp3:
    schema_profile = exp_3_df[exp_3_df['Stimulus'] == s].iloc[:, 1:6].values[0].tolist()
    x_original3.append(schema_profile)
for s in stimuli_exp4:
    schema_profile = exp_4_df[exp_4_df['Stimulus'] == s].iloc[:, 1:6].values[0].tolist()
    x_original4.append(schema_profile)

# LDA with participant data with IS
clf = QuadraticDiscriminantAnalysis()
clf.fit(x_original3, y)
y_pred3 = clf.predict(x_original3)
print('accuracy original exp3:', accuracy_score(y_true=y, y_pred=y_pred3))

clf.fit(x_original4, y)
y_pred4 = clf.predict(x_original4)
print('accuracy original exp4:', accuracy_score(y_true=y, y_pred=y_pred4))



accuracy original exp3: 0.6875
accuracy original exp4: 0.71875


In [138]:
for i in range(len(x_original4)):
    print(stimuli_exp4[i])
    print(x_original4[i])
    print("")

be at attention
[3.4, 5.1, 4.53, 3.3, 3.5]

be distinguished in several sports
[2.2, 3.0, 4.43, 3.73, 4.67]

to hold firm
[5.47, 3.67, 4.73, 3.3, 4.0]

don't allow such treatment
[5.9, 2.6, 3.03, 3.13, 4.03]

to pass the test of time
[4.03, 2.4, 3.4, 2.73, 4.2]

united we are strong
[4.33, 3.13, 4.93, 4.27, 5.73]

we are backed up by 30 years experience
[2.5, 2.73, 3.0, 3.13, 5.2]

let the issue remain as is
[3.37, 2.47, 2.97, 2.47, 2.87]

leave the mixture undisturbed
[2.97, 2.43, 3.6, 2.6, 2.33]

have a dale with someone who didn 't show up
[3.93, 2.8, 2.27, 3.87, 4.23]

he measures six-foot nine
[1.6, 5.73, 2.73, 2.3, 2.4]

the clock is on the mantle
[1.17, 3.23, 3.6, 2.77, 3.1]

one-night fling
[2.7, 2.03, 2.1, 3.13, 3.43]

to be in the position to make a profit
[2.03, 2.73, 3.2, 4.27, 3.9]

to be in someone else's shoes
[2.57, 2.63, 2.1, 4.13, 4.67]

on the witness platform
[3.17, 3.4, 2.6, 4.83, 4.1]

be in aw
[2.23, 2.47, 2.57, 3.43, 4.37]

the police told them to get back
[5.43

In [139]:
for i in range(len(x_original3)):
    print(stimuli_exp3[i])
    print(x_original3[i])
    print("")

stand at attention
[4.0, 6.08, 5.0, 3.08, 2.08]

stand out in several sports
[2.0, 3.54, 2.83, 5.04, 3.42]

to stand firm
[6.21, 4.13, 5.04, 3.75, 2.83]

don't stand for such treatment
[5.54, 2.21, 3.04, 3.13, 3.08]

to stand the test of time
[4.88, 2.38, 3.92, 3.0, 4.08]

united we stand
[4.79, 4.21, 4.25, 4.78, 6.33]

we stand on 30 years of experience
[2.46, 2.63, 2.92, 3.43, 4.58]

let the issue stand
[2.83, 1.92, 2.08, 3.04, 2.96]

let the mixture stand
[1.54, 2.08, 3.17, 1.88, 2.0]

get stood up for a date
[1.83, 3.17, 3.04, 2.29, 2.17]

he stands six-foot nine
[2.29, 6.5, 3.54, 1.96, 2.0]

the clock stands on the mantle
[2.0, 4.38, 4.46, 2.13, 2.42]

one-night stand
[1.63, 1.46, 1.29, 2.17, 3.04]

to stand to profit
[2.04, 1.75, 1.79, 1.74, 3.54]

to stand in someone else's shoes
[2.25, 3.58, 2.96, 4.13, 4.7]

on the witness stand
[1.83, 2.08, 1.5, 3.63, 2.96]

stand in awe
[2.33, 3.75, 3.29, 3.65, 3.5]

the police told them to stand back
[3.63, 3.42, 2.17, 3.46, 2.74]

stand by